# Imports and Variables

In [487]:
from xml.dom import minidom
import io
from PIL import Image  # https://pillow.readthedocs.io/en/4.3.x/
import requests  # http://docs.python-requests.org/en/master/
import re

s = 'Residential / Home'
s.split(" / ")
k = 0
pais = estado = cidade = bairro = endereco = cep = ref = titulo = aluguelVenda = 0
tipo_prop = descricao = area = quartos = banheiros = suites = garagem = 0
location_list = detail_list = list()
p = s = c = n = a = 0

# Functions

In [488]:
# example image url: https://m.media-amazon.com/images/S/aplus-media/vc/6a9569ab-cb8e-46d9-8aea-a7022e58c74a.jpg
def download_image(url, image_file_path):
    r = requests.get(url, timeout=4.0)
    if r.status_code != requests.codes.ok:
        assert False, 'Status code error: {}.'.format(r.status_code)

    with Image.open(io.BytesIO(r.content)) as im:
        im.save(image_file_path)
    print(f"Image downloaded from url: {format} and saved to: {image_file_path}")


def en_to_pt_resCom(word):
    word = word.split(" / ")
    # Translate the First Word
    if word[0] == "Residential":
        word[0] = 'Residencial'
    else:
        word[0] = 'Comercial'

    # Translate the second Word
    if word[1] == 'Home':
        word[1] = 'Casa'

    elif word[1] == 'Land Lot':
        word[1] = 'Terreno'

    elif word[1] == 'Farm Ranch':
        word[1] = 'Chácaras'

    elif word[1] == 'Apartment':
        word[1] = 'Apartamento'

    else:
        word[1] = word[1]

    return " / ".join(word)


def en_to_pt_rentSale(word):
    if word == 'For Rent':
        word = 'Aluguel'
    else:
        word = 'Venda'
    return word

# Opening the XmL documment

In [489]:
with open("XML-IMO.xml", 'r', encoding='utf-8') as f:
    xml = minidom.parse(f)
    Listing = xml.getElementsByTagName("Listing")

# First Loop To Find the Location INFO

In [490]:
    for count, tag in enumerate(Listing):
        for count_inside, local in enumerate(tag.getElementsByTagName("Location")):
            if not count_inside:
                ref = (f"referencia: {tag.getElementsByTagName('ListingID')[0].firstChild.data}")
                titulo = (f"Titulo/descricao: {tag.getElementsByTagName('Title')[0].firstChild.data}")
                aluguelVenda = (f"Venda/Aluguel: {en_to_pt_rentSale(tag.getElementsByTagName('TransactionType')[0].firstChild.data)}")
                pais = (f"Pais: {local.getElementsByTagName('Country')[0].firstChild.data}")
                endereco = (f"Endereco/Rua: {local.getElementsByTagName('Address')[0].firstChild.data}")
                estado = (f"Estado: {local.getElementsByTagName('State')[0].firstChild.data}")
                cidade = (f"Cidade: {local.getElementsByTagName('City')[0].firstChild.data}")
                bairro = (f"Bairro: {local.getElementsByTagName('Neighborhood')[0].firstChild.data}")
                try:
                    cep = (f"CEP: {local.getElementsByTagName('PostalCode')[0].firstChild.data}")
                except AttributeError:
                    cep = (f"CEP: --------")
                location_list.append([ref, titulo, cep, endereco, bairro, cidade, estado, pais, aluguelVenda])


# Second loop to find the Listing DETAILS

In [491]:
    for count, tag in enumerate(Listing):
        for count_detail, detail in enumerate(tag.getElementsByTagName("Details")): #ok
            tipo_prop = (f"Finalidade/Imovel: {en_to_pt_resCom(detail.getElementsByTagName('PropertyType')[0].firstChild.data)}")
            descricao = (f"Descricao: {re.sub(' +', ' ', detail.getElementsByTagName('Description')[0].firstChild.wholeText).replace('n', '')}") # alert
            area = (f"Area: {detail.getElementsByTagName('ConstructedArea')[0].firstChild.data}") # alert
            quartos = (f"Quartos: {detail.getElementsByTagName('Bedrooms')[0].firstChild.data}") # alert
            banheiros = (f"Banheiros: {detail.getElementsByTagName('Bathrooms')[0].firstChild.data}") # alert
            suites = (f"Suites: {detail.getElementsByTagName('Suites')[0].firstChild.data}") # alert
            try:
                garagem = (f"Vagas na Garegem: {detail.getElementsByTagName('Garage')[0].firstChild.data}")
            except (AttributeError, IndexError):
                garagem = ("Não tem garagem")

            detail_list.append([descricao, tipo_prop, quartos, suites, banheiros, garagem, area])

            # print((detail.getElementsByTagName('ListPrice')[0].firstChild.data))

            # if len(detail.getElementsByTagName('rentalPrice')) == 1:
            #     print(((detail.getElementsByTagName('rentalPrice'))[0].firstChild.data), 'Aluga')
            # else:
            #     if len(detail.getElementsByTagName('ListPrice')) == 1:
            #         print(((detail.getElementsByTagName('ListPrice'))[0].firstChild.data), 'venda',count)

            # garage, RentalPrice/ListPrice

            #tipo_prop = descricao = area = quartos = banheiros = suites = garagem = 0